In [4]:
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
from qiskit.visualization import *
from qiskit_ibm_runtime import *
import numpy as np
from qiskit import *
from qiskit_aer import *
from scipy.fft import fft
from scipy.stats import entropy

# Function to generate QRNG data
def generate_qrng_data(num_bits):
    simulator = AerSimulator()
    qrng_data = []

    for _ in range(num_bits):
        circ = QuantumCircuit(1, 1)
        circ.h(0)  # Apply Hadamard gate
        circ.measure(0, 0)  # Measure the qubit

        compiled_circ = transpile(circ, simulator)
        result = simulator.run(compiled_circ, shots=1).result()
        counts = result.get_counts()
        qrng_data.append(int(list(counts.keys())[0]))

    return np.array(qrng_data)

# Function to create Toeplitz matrix
def create_toeplitz(first_row, first_column):
    n = len(first_row)
    m = len(first_column)
    toeplitz_matrix = np.zeros((m, n), dtype=int)

    for i in range(m):
        for j in range(n):
            if i <= j:
                toeplitz_matrix[i, j] = first_row[j - i]
            else:
                toeplitz_matrix[i, j] = first_column[i - j]

    return toeplitz_matrix

# Function to calculate Min-Entropy
def min_entropy(data):
    counts = np.bincount(data)
    probabilities = counts / len(data)
    max_prob = np.max(probabilities)
    return -np.log2(max_prob)

# Function to calculate Collision Entropy
def collision_entropy(data):
    counts = np.bincount(data)
    probabilities = counts / len(data)
    return -np.log2(np.sum(probabilities ** 2))

# Function to perform NIST-like frequency test
def nist_frequency_test(data):
    proportion = np.sum(data) / len(data)
    return abs(proportion - 0.5) < 0.01  # Checks if close to uniform (within 1%)

# Parameters
num_bits = 1024  # Number of QRNG bits to generate
block_size = 128  # Size of each block for Toeplitz transformation

# Step 1: Generate QRNG data
qrng_data = generate_qrng_data(num_bits)

# Step 2: Define Toeplitz matrices
first_row = np.random.randint(0, 2, block_size)
first_column = np.random.randint(0, 2, block_size)
toeplitz_matrix = create_toeplitz(first_row, first_column)

# Step 3: Apply Toeplitz transformation
blocks = [qrng_data[i:i + block_size] for i in range(0, len(qrng_data), block_size)]
transformed_data = np.concatenate([
    np.dot(toeplitz_matrix, block[:block_size]) % 2 for block in blocks
])

# Step 4: Apply FFT Toeplitz transformation
fft_toeplitz_matrix = fft(toeplitz_matrix)
fft_transformed_data = np.concatenate([
    np.round(np.real(ifft(fft_toeplitz_matrix * fft(block[:block_size])))) % 2 for block in blocks
]).astype(int)

# Fix for Entropy Calculation: Ensure all data is 1D binary
transformed_data = transformed_data.flatten().astype(int)
fft_transformed_data = fft_transformed_data.flatten().astype(int)

# Step 5: Calculate entropy metrics
original_shannon_entropy = entropy(np.bincount(qrng_data) / len(qrng_data), base=2)
transformed_shannon_entropy = entropy(np.bincount(transformed_data) / len(transformed_data), base=2)
fft_transformed_shannon_entropy = entropy(np.bincount(fft_transformed_data) / len(fft_transformed_data), base=2)

original_min_entropy = min_entropy(qrng_data)
transformed_min_entropy = min_entropy(transformed_data)
fft_transformed_min_entropy = min_entropy(fft_transformed_data)

original_collision_entropy = collision_entropy(qrng_data)
transformed_collision_entropy = collision_entropy(transformed_data)
fft_transformed_collision_entropy = collision_entropy(fft_transformed_data)

# Step 6: NIST frequency test
original_nist = nist_frequency_test(qrng_data)
transformed_nist = nist_frequency_test(transformed_data)
fft_transformed_nist = nist_frequency_test(fft_transformed_data)

# Results
print("--- QRNG Data Analysis ---")
print(f"Original Shannon Entropy: {original_shannon_entropy:.4f}")
print(f"Transformed Shannon Entropy: {transformed_shannon_entropy:.4f}")
print(f"FFT Transformed Shannon Entropy: {fft_transformed_shannon_entropy:.4f}\n")

print(f"Original Min-Entropy: {original_min_entropy:.4f}")
print(f"Transformed Min-Entropy: {transformed_min_entropy:.4f}")
print(f"FFT Transformed Min-Entropy: {fft_transformed_min_entropy:.4f}\n")

print(f"Original Collision Entropy: {original_collision_entropy:.4f}")
print(f"Transformed Collision Entropy: {transformed_collision_entropy:.4f}")
print(f"FFT Transformed Collision Entropy: {fft_transformed_collision_entropy:.4f}\n")

print(f"Original NIST Frequency Test Passed: {original_nist}")
print(f"Transformed NIST Frequency Test Passed: {transformed_nist}")
print(f"FFT Transformed NIST Frequency Test Passed: {fft_transformed_nist}")

--- QRNG Data Analysis ---
Original Shannon Entropy: 0.9991
Transformed Shannon Entropy: 0.9989
FFT Transformed Shannon Entropy: 0.9999

Original Min-Entropy: 0.9502
Transformed Min-Entropy: 0.9447
FFT Transformed Min-Entropy: 0.9871

Original Collision Entropy: 0.9982
Transformed Collision Entropy: 0.9978
FFT Transformed Collision Entropy: 0.9999

Original NIST Frequency Test Passed: False
Transformed NIST Frequency Test Passed: False
FFT Transformed NIST Frequency Test Passed: True
